### Hypothesis testing
Elements of Data Science

## Hypothesis Testing Learning Goals
Develop and test an hypothesis
- Hypothesis
    - testable hypothesis
    - statistic
- Simulation: Sample the distribution
    - Repeat and collect outcomes
    - Iteration: 
        `for i in np.arange(samples)`
- Examine resulting distribution of outcomes
    - Probability distribution
    - Uncertainty
- p-test

In [2]:
import numpy as np
from datascience import *

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# Fix for datascience plots
import collections as collections
import collections.abc as abc
collections.Iterable = abc.Iterable

## Hypothesis Testing

#### Ground Hog


In [4]:
groundhogdata = Table.read_table('Groundhogdata/summarizedGroundhogData_20210326.csv')
groundhogdata

hogID,year,shadowPres,dateSample,julDay,GDD,cGDD,numObs,meanJulSpringDay,sdJulSpringDay,seJulSpringDay,earlyOrLate
BC,2018,yes,2018-04-11,101,3.05556,127.222,30,85.8667,12.4367,2.27062,late
BC,2019,no,2019-04-06,96,8.33333,123.889,30,85.8667,12.4367,2.27062,late
BECK,2006,yes,2006-02-15,46,7.22222,127.006,30,72.4333,14.6586,2.67628,early
BECK,2007,no,2007-03-14,73,14.1667,129.167,30,72.4333,14.6586,2.67628,late
BECK,2008,no,2008-03-17,77,0.833333,128.611,30,72.4333,14.6586,2.67628,late
BECK,2009,yes,2009-03-10,69,12.2222,130,30,72.4333,14.6586,2.67628,early
BECK,2010,no,2010-03-21,80,7.22222,129.444,30,72.4333,14.6586,2.67628,late
BECK,2011,no,2011-03-20,79,7.77778,133.611,30,72.4333,14.6586,2.67628,late
BECK,2012,no,2012-03-07,67,13.3333,129.444,30,72.4333,14.6586,2.67628,early
BECK,2013,no,2013-03-30,89,3.88889,128.611,30,72.4333,14.6586,2.67628,late


In [5]:
yes_late = groundhogdata.where('shadowPres', 'yes').where('earlyOrLate', 'late')
no_early = groundhogdata.where('shadowPres', 'no').where('earlyOrLate', 'early')

correct_tbl = yes_late.append(no_early)
correct_tbl.group('hogID')

hogID,count
BC,1
BECK,1
CACK,6
CECK,4
CID,2
CS,8
DKDV,4
EE,7
FCFE,5
FDG,4


In [6]:
percent_correct = correct_tbl.num_rows / groundhogdata.num_rows * 100
percent_correct


51.132075471698116

In [7]:
shadow_options = make_array('yes', 'no')
spring_options = make_array('late', 'early')
num_observations = groundhogdata.num_rows
groundhogdata.num_rows

530

In [8]:
np.arange(num_observations)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [ ]:
right = 0 
wrong = 0

for obs in np.arange(num_observations):
    shadow = np.random.choice(shadow_options)
    spring = np.random.choice(spring_options)
    #print(obs,shadow,spring)
    if shadow == 'yes' and spring == 'late':
        right = right + 1
    elif shadow == 'no' and spring == 'early':
        right += 1
    else:
        wrong += 1

simulated_num_correct = right / num_observations
simulated_num_correct

In [ ]:
def sim_ground(repeats):
    correct_obs = []
    for i in np.arange(repeats):
        right = 0 
        wrong = 0
        for obs in range(num_observations):
            shadow = np.random.choice(shadow_options)
            spring = np.random.choice(spring_options)
            if shadow == 'yes' and spring == 'late':
                right += 1
            elif shadow == 'no' and spring == 'early':
                right += 1
            else:
                wrong += 1
        simulated_num_correct = right / num_observations
        correct_obs.append(simulated_num_correct)
    return correct_obs        

In [ ]:
plt.hist(sim_ground(1000),bins=np.arange(0.4,0.6,.01),color = "skyblue", ec="red")
plt.title('Simulated Groundhog outcomes')
plt.savefig('sim_ground.png')
plt.show()

In [ ]:
hogsimdata = Table().with_columns("Correct",sim_ground(1500))
hogsimdata

In [ ]:
hogsimdata.hist("Correct",bins=np.arange(0.4,0.6,.01))
plt.scatter(percent_correct/100, 0, color='green', s=200);
plt.title('Simulated Groundhog outcomes')
plt.savefig('sim_ground_correct.png')
plt.show()

In [ ]:
np.count_nonzero(hogsimdata.column('Correct') >= percent_correct/100) / 1500

#### Essex Ed

In [ ]:
groundhogdata.where('hogID',"EE").num_rows

In [ ]:
correct_tbl.where('hogID',"").num_rows

In [ ]:
num_correct = correct_tbl.where('hogID',"PYPL").num_rows / groundhogdata.where('hogID',"PYPL").num_rows * 100
num_correct

In [ ]:
hogsimdata.hist(0,bins=np.arange(0.4,0.6,.01))
plt.scatter(num_correct/100, 0, color='red', s=200);
plt.title('Simulated Groundhog outcomes')
plt.savefig('sim_ground_correct.png')
plt.show()

In [ ]:
np.count_nonzero(hogsimdata.column('Correct') >= num_correct/100) / 1500

### Central Limit Theorem

In [ ]:
import numpy.random as np
import seaborn as sns
import matplotlib.pyplot as plt
population_size = 1000000
population = np.rand(1000000)

In [ ]:
number_of_samples = 10000
sample_means = np.rand(number_of_samples)
sample_size = 1

In [ ]:
c = np.rand(number_of_samples)
for i in range(0,number_of_samples):
 c = np.randint(1,population_size,sample_size)
 sample_means[i] = population[c].mean()

In [ ]:
plt.subplot(1,2,1)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
sns.distplot(sample_means,bins=int(180/5),hist = True,kde = False)
plt.title('Histogram of Sample mean',fontsize=20)
plt.xlabel('Sample mean',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.subplot(1,2,2)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
sns.distplot(sample_means,hist = False,kdeplot = True)
plt.title('Density of Sample mean',fontsize=20)
plt.xlabel('Sample mean',fontsize=20)
plt.ylabel('Density',fontsize=20)
plt.subplots_adjust(bottom=0.1, right=2, top=0.9)

### Testing differences

In [ ]:
def diff_n(values, n):
    return np.array(values)[n:] - np.array(values)[:-n]

diff_n(make_array(1, 10, 100, 1000, 10000), 2)

In [ ]:
!pip install meteostat
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily

In [ ]:
# Set time period
start = datetime(2021, 1, 1)
end = datetime(2021, 12, 31)
# Create Point for SERC, Philadelphia
location = Point(39.9816, -75.153, 70)
# Get daily data for 2021
data = Daily(location, start, end)
data = data.fetch()['tavg'].values # Values in Celsius
data = (data*9/5)+32

In [ ]:
differences = diff_n(rates, years)